In [2]:
from k_means_clustering import kmeans

In [5]:
output_img = kmeans('D:/work/kmeans/images/2024_01_30_cutted.tif', 4, "D:/work/kmeans/result_images/image3.geotif")

C:\Users\AK\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
